# Semantic Search Through Resumes for HR Tech Startups Using Qdrant

## Utilise Qdrant, Langchain and OpenAI to narrow down on top resumes for the job


In [1]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
import getpass
import os


os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [2]:
loader = PyPDFDirectoryLoader("../data/INFORMATION-TECHNOLOGY")
docs = loader.load()

In [3]:
print(len(docs))
print(docs)

247
[Document(page_content="INFORMATION TECHNOLOGY SPECIALIST/SYSTEM ANALYSIS\nSummary\nI have over 10 years of professional service in Information Technology Support and Technical Operations in working with the Federal Government\nand private sectors. I have a broad knowledge and expertise in strategic planning, IT Business Systems, Network Operations, IT Security and\nSystem Analysis. My goal is to secure permanent employment within a outstanding organization that offer opportunities for growth and\nadvancement, while implementing a high degree of professionalism, enthusiasm, initiative on a daily basis.\nHighlights\nActive Top Security Clearance/SCI\nCertified Cisco Network Associate\nCertified CompTIA Security+\nMicrosoft SharePoint 2010 Train\nExperience\nCompany Name\n \nFebruary 2015\n \nto \nMay 2016\n \nInformation Technology Specialist/System Analysis\n \nCity\n \n, \nState\nServe as IT System Analyst; assist with business process improvement efforts, responsible for the phys

4881